In [ ]:
## global_time 추가, OUPUT_STATION의 tuple 형태, INPUT_STATION의 tuple 형태
def CAStar(graph,AGV_LIST,INPUT_STATION,OUTPUT_STATION,reservation_table,global_time):
    agv_avail = getAvailAGV(AGV_LIST,battery_threshold)
    total_cost = 0
    while len(agv_avail)!=0 and jobQ.job_number!=0:
        job = jobQ.getJob()
        print(job)
#         print(job['station_dest'])
        agv_avail = getAvailAGV(AGV_LIST, battery_threshold)
        for agv in agv_avail:
            agv.calculate_bid(job,warehouse,INPUT_STATION[0].coordinates,OUTPUT_STATION[0].coordinates)
            agv.bid_list.append(agv.calculate_bid(job,warehouse,INPUT_STATION[0].coordinates,OUTPUT_STATION[0].coordinates))
        for agv_ in agv_avail:
#             print(agv_bid_list)
            agv_bid_sort = sorted(agv_.bid_list,key = lambda x:x[0], reverse = False)
#             print(agv_bid_sort)
        bid_compare_battery=[]
        for i in range(len(agv_bid_sort)):
            if agv_bid_sort[i][1]==agv_bid_sort[0][1]:
                bid_compare_battery.append((agv_bid_sort[i]))
        bid_compare_battery_sort = sorted(bid_compare_battery, key= lambda x:x[1].battery, reverse =True)

        reset_graph(graph)
         
        if job['inout']=='in':
            castar_path = findPathAstar(graph,bid_compare_battery_sort[0][1],bid_compare_battery_sort[0][1].current_location,INPUT_STATION[0].coordinates,reservation_table)+findPathAstar(graph,bid_compare_battery_sort[0][1],INPUT_STATION[0].coordinates,Rack_enterence[job['station_dest']],reservation_table)
        else:
            castar_path = findPathAstar(graph,bid_compare_battery_sort[0][1],bid_compare_battery_sort[0][1].current_location,Rack_enterence[job['station_dest']],reservation_table)+findPathAstar(graph,bid_compare_battery_sort[0][1],Rack_enterence[job['station_dest']],OUTPUT_STATION[0].coordinates,reservation_table)
        
        
        
        if castar_path == None:
            return -1
        for time,value in enumerate(castar_path):
#             reservation_table[hash(value.depth, value.id)] = True 
            reservation_table[(value.depth+global_time, value.id)] = True  
        target_agv = bid_compare_battery_sort[0][0]
        select_agv_list.append(target_agv)
        target_agv.get_job(job,castar_path)
        print(" target_agv : {}, target_agv.battery: {}, destination : {}, castar_path : {}, available:{}".format(target_agv,target_agv.battery,job['station_dest'],[i.coordinates for i in castar_path],target_agv.available))
#         ### path 찍어보기 
#         for move in castar_path:
#             print(move.coordinates)
#         print(reservation_table)
        for agv in AGV_LIST:
                agv.clear_bid_list(AGV_LIST)
        agv_avail = getAvailAGV(AGV_LIST,battery_threshold)
        for agv in AGV_LIST:
            total_cost += len(agv.walking_path)
            return total_cost